In [ ]:
!pip install pdfplumber


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pdfplumber
import os
import csv
import logging

In [3]:
from dotenv import load_dotenv

load_dotenv()

input_path = os.getenv("INPUT_PATH" | "./data/sample.pdf")
output_path = os.getenv("OUTPUT_PATH" | "./data/decypted")
password = os.getenv("PASSWORD" | "password")

In [5]:
def extract_tables_from_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:

            # Define the output CSV file path
            month_number = pdf_path.split('/')[-1].split('_')[1]
            csv_filename = f"{month_number}.csv"
            tables = []
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    tables.append(table)
            return tables, csv_filename
    except Exception as e:
        logging.error(f"Error occured extracting from pdf {pdf_path}",{e})

In [6]:
def save_table_to_csv(tables, csv_filename, output_dir):
    try:
        csv_path = os.path.join(output_dir, csv_filename)
        with open(csv_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            for table in tables:
                for row in table:
                    writer.writerow(row)
        logging.info(f"Saved CSV: {csv_path}")
    except Exception as e:
        logging.error(f"Error occured saving table to csv {csv_filename}",{e})

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

# Decrypt the PDF
def decrypt_pdf(input_path, output_path, password):
    reader = PdfReader(input_path)
    if reader.is_encrypted:
        reader.decrypt(password)
    
    writer = PdfWriter()
    for page in reader.pages:
        writer.add_page(page)
    
    with open(output_path, "wb") as output_file:
        writer.write(output_file)

In [7]:
def process_pdfs_in_directory(directory_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    # List all PDF files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            pdf_file_path = os.path.join(directory_path, filename)
            # output_dir = os.path.join(output_dir,
            # Decrypt the PDF if necessary
            decrypt_pdf(pdf_file_path, output_path, password)
            
            # Extract tables from the PDF and save them to CSV
            logging.info(f"Processing: {pdf_file_path}")
            tables, csv_filename = extract_tables_from_pdf(pdf_file_path)
            if tables:
                save_table_to_csv(tables, f"{csv_filename}", output_dir)
            else:
                logging.info(f"No tables found in: {pdf_file_path}")

In [11]:
pdf_directory_path = './data/decrypted/'
output_directory_path = './data/output/'
process_pdfs_in_directory(pdf_directory_path, output_directory_path)